In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
data = pd.read_excel("PTA.xlsx")
pd.set_option("max_columns", 9999)
np.random.seed(7)

# Data processing

In [2]:
#select variables available pre-aspiration attempt:

columns = ['Age', 'Gender', 'Duration Sxs (days)', 'Fever', 'Sore Throat', 'Otalgia', 
           'Trismus','Cough', 'Dysphagia', 'Anorexia','Worsening of Symptoms', "Pus", 'WBC ']

# drop the 1 missing age row:

data['Age'] = data['Age'].dropna()

In [3]:
#convert continuous age variable to categorical:

for idx,row in data.iterrows():
    if row['Age'] < 18:
        data.loc[idx, 'Age_cat'] = "Teenager"
    elif row['Age'] >= 18 and row['Age'] < 30:
        data.loc[idx, 'Age_cat'] = "Young Adult"
    elif row['Age'] >= 30 and row['Age'] < 50:
        data.loc[idx, 'Age_cat'] = "Adult"
    elif row['Age'] >50:
        data.loc[idx, 'Age_cat'] = "50+"

def create_dummies(df,column_name):
    """Create Dummy Columns (One Hot Encoding) from a single Column

    Usage
    ------
    """
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

data = create_dummies(data, 'Age_cat').drop(columns='Age')

In [4]:
#convert gender to numerical:

for index, row in data.iterrows():
    if row['Gender'] == "M":
        data.loc[index,'Gender'] = 1
    else:
        data.loc[index,'Gender'] = 0

In [5]:
#drop NR value from Duration of symptoms:

data =data [data['Duration Sxs (days)']!= "NR"] 

In [6]:
# deal with missing values for WBC count:

import numpy as np
for idx,row in data.iterrows():
    if row['WBC '] == 'Not Performed':
        data.loc[idx, 'WBC ']=np.nan
    elif row['WBC '] == 0:
        data.loc[idx, 'WBC ']=np.nan
        
data['WBC '] = data['WBC '].astype("float64")

In [7]:
#convert to numeric, drop rows with missing data
data['Duration Sxs (days)'] = data['Duration Sxs (days)'].astype("float64")
data = data[(data['Tonsillectomy'] != "Previous") & (data['Tonsillectomy'] != "-")]
data['Tonsillectomy'] = data['Tonsillectomy'].astype('float64')
data = data[['Gender', 'Duration Sxs (days)', 'Fever', 'Otalgia', 
           'Trismus','Cough', 'Dysphagia', 'Anorexia', 'Worsening of Symptoms', 'WBC ', 'Age_cat_50+', 
            'Age_cat_Teenager', 'Age_cat_Adult', 'Age_cat_Young Adult', 'Pus', 'Tonsillectomy']]

data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 656 entries, 0 to 1055
Data columns (total 16 columns):
Gender                   656 non-null int64
Duration Sxs (days)      656 non-null float64
Fever                    656 non-null float64
Otalgia                  656 non-null float64
Trismus                  656 non-null float64
Cough                    656 non-null float64
Dysphagia                656 non-null float64
Anorexia                 656 non-null float64
Worsening of Symptoms    656 non-null float64
WBC                      656 non-null float64
Age_cat_50+              656 non-null uint8
Age_cat_Teenager         656 non-null uint8
Age_cat_Adult            656 non-null uint8
Age_cat_Young Adult      656 non-null uint8
Pus                      656 non-null float64
Tonsillectomy            656 non-null float64
dtypes: float64(11), int64(1), uint8(4)
memory usage: 69.2 KB


656 rows remain after dropping all with missing values

## Correlations

In [8]:
corr = data[['Gender', 'Duration Sxs (days)', 'Fever', 'Otalgia', 
           'Trismus','Cough', 'Dysphagia', 'Anorexia', 'Worsening of Symptoms', 'Age_cat_50+', 
            'Age_cat_Teenager', 'Age_cat_Adult', 'Age_cat_Young Adult', 'Pus']].corr()
corr['Pus'].sort_values(ascending=False)


Pus                      1.000000
Trismus                  0.432583
Worsening of Symptoms    0.273227
Otalgia                  0.109684
Duration Sxs (days)      0.102332
Age_cat_Young Adult      0.065727
Dysphagia                0.061706
Gender                   0.056037
Anorexia                 0.014675
Age_cat_50+             -0.001339
Fever                   -0.008247
Age_cat_Adult           -0.019917
Cough                   -0.053863
Age_cat_Teenager        -0.071192
Name: Pus, dtype: float64

In [9]:
pd.pivot_table(data, index='Trismus', values = 'Pus')

,Pus
Trismus,
0.0,0.367041
1.0,0.794344


In [10]:
pd.pivot_table(data, index='Otalgia', values = 'Pus')

,Pus
Otalgia,
0.0,0.557971
1.0,0.665789


Trismus looks much more predictive of successful aspiration than otalgia

# Models

### Random Forest


In [11]:
# feature selection:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV

def RFECV_selection(df, features):
    all_X = df[features]
    all_y = df['Pus']
    rf = RandomForestClassifier(random_state=1)
    rfecv = RFECV(rf, cv=10)
    rfecv.fit(all_X, all_y)
    print("best features \n _________________ \n", 
          list(all_X.columns[rfecv.support_]))
    return list(all_X.columns[rfecv.support_])

features = ['Gender', 'Duration Sxs (days)', 'Fever', 'Otalgia', 
           'Trismus','Cough', 'Dysphagia', 'Anorexia', 'Worsening of Symptoms', 'Age_cat_50+', 
            'Age_cat_Teenager', 'Age_cat_Adult', 'Age_cat_Young Adult']

best_features = RFECV_selection(data, features)

best features 
 _________________ 
 ['Duration Sxs (days)', 'Trismus', 'Worsening of Symptoms']


Interestingly, eliminates all but duration of symptoms, trismus, and worsening of symptoms. However, in the interest of interpretability and broader applicability (beyond just patients already selected for specialist evaluation of PTA), we opted to include all features in the model

In [12]:
# training the model:

# create training and holdout sets:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

X_train, X_test, y_train, y_test = train_test_split(data[features], data['Pus'], test_size=0.3, random_state=1)
X_train = normalize(X_train)
X_test = normalize(X_test)

In [13]:
# Grid search for parameter optimization
# this may take a minute to run...

from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
grid = GridSearchCV(rf, {
            "n_estimators": [10, 50, 100],
            "criterion": ["entropy", "gini"],
            "max_depth": [2, 5, 10, 20],
            "max_features": ["log2"],
            "min_samples_leaf": [1, 5, 8, 15],
            "min_samples_split": [2, 3, 5, 9, 15]
        }, cv = 10)
grid.fit(X_train, y_train)
best_rf = grid.best_estimator_
print(grid.best_params_)

{'criterion': 'gini', 'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 15, 'min_samples_split': 5, 'n_estimators': 50}


#### Evaluating model performance

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

best_rf.fit(X_train, y_train)
predictions = best_rf.predict(X_test)
predictions_train = best_rf.predict(X_train)
accuracy_test = accuracy_score(y_test, predictions)
accuracy_train = accuracy_score(y_train, predictions_train)
print(accuracy_test, accuracy_train)
print(precision_recall_fscore_support(y_test, predictions))
print(classification_report(y_test, predictions))

0.761421319797 0.762527233115
(array([ 0.68627451,  0.78767123]), array([ 0.53030303,  0.8778626 ]), array([ 0.5982906 ,  0.83032491]), array([ 66, 131]))
             precision    recall  f1-score   support

        0.0       0.69      0.53      0.60        66
        1.0       0.79      0.88      0.83       131

avg / total       0.75      0.76      0.75       197



Accuracy on the test and training sets are similar, suggesting no overfitting.

In [15]:
#randomly divide the data into training and test sets 100 times, train and test the model, and average the accuracy
rf_accuracy = []
rf_accuracy_train = []

for i in range (500):
    
    X_train, X_test, y_train, y_test = train_test_split(data[features], data['Pus'], 
                                                        test_size=0.30)
    X_train = normalize(X_train)
    X_test = normalize(X_test)
    
    best_rf.fit(X_train, y_train)
    predictions = best_rf.predict(X_test)
    predictions_train = best_rf.predict(X_train)
    rf_accuracy.append(accuracy_score(y_test, predictions))
    rf_accuracy_train.append(accuracy_score(y_train, predictions_train))
    
    
print(np.mean(rf_accuracy))
print(np.mean(rf_accuracy_train))

0.726192893401
0.780091503268


### Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
grid = GridSearchCV(lr, {"solver": ["newton-cg", "lbfgs", "liblinear"]}, cv=10)
grid.fit(X_train, y_train)
best_lr = grid.best_estimator_
print(grid.best_score_)

0.692810457516


In [17]:
#randomly divide the data into training and test sets 100 times, train and test the model, and average the accuracy
accuracy = []
accuracy_train = []
for i in range (100):
    
    X_train, X_test, y_train, y_test = train_test_split(data[features], data['Pus'], 
                                                        test_size=0.30, random_state=np.random.randint(1, 100))
    X_train = normalize(X_train)
    X_test = normalize(X_test)
    
    best_lr.fit(X_train, y_train)
    predictions = best_lr.predict(X_test)
    predictions_train = best_lr.predict(X_train)
    accuracy.append(accuracy_score(y_test, predictions))
    accuracy_train.append(accuracy_score(y_train, predictions_train))
print(np.mean(accuracy))
print(np.mean(accuracy_train))

0.690355329949
0.692549019608


Logistic regression is less accurate than Random Forest

### K Nearest Neighbors

In [18]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
grid = GridSearchCV(knn, {
            'n_neighbors': range(1,20,2),
            'weights': ['distance', 'uniform'],
            'algorithm': ['ball_tree', 'kd_tree', 'brute'],
            'p': [1,2]
        }, cv=10)
grid.fit(X_train, y_train)
best_knn = grid.best_estimator_
print(grid.best_score_)
print(grid.best_params_)

0.705882352941
{'algorithm': 'brute', 'n_neighbors': 13, 'p': 1, 'weights': 'uniform'}


In [19]:
#evaluate performance:

best_knn.fit(X_train, y_train)
predictions = best_knn.predict(X_test)
predictions_train = best_knn.predict(X_train)
accuracy_test = accuracy_score(y_test, predictions)
accuracy_train = accuracy_score(y_train, predictions_train)
print(accuracy_test, accuracy_train)

0.741116751269 0.723311546841


In [20]:
#randomly divide the data into training and test sets 100 times, train and test the model, and average the accuracy
accuracy = []
accuracy_train = []
for i in range (100):
    
    X_train, X_test, y_train, y_test = train_test_split(data[features], data['Pus'], 
                                                        test_size=0.30)
    X_train = normalize(X_train)
    X_test = normalize(X_test)

    best_knn.fit(X_train, y_train)
    predictions = best_lr.predict(X_test)
    predictions_train = best_lr.predict(X_train)
    accuracy.append(accuracy_score(y_test, predictions))
    accuracy_train.append(accuracy_score(y_train, predictions_train))
    
print(np.mean(accuracy))
print(np.mean(accuracy_train))

0.699492385787
0.686710239651


Better than LR, but still not as good as random forest

### Neural Network

In [40]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPClassifier(max_iter = 2000)
mlp.fit(X_train, y_train)
predictions = mlp.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(accuracy, "\n\n", classification_report(y_test, predictions))


0.771573604061 

              precision    recall  f1-score   support

        0.0       0.70      0.56      0.62        66
        1.0       0.80      0.88      0.84       131

avg / total       0.76      0.77      0.76       197



In [22]:
#evaluate optimal parameters:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPClassifier()
grid = GridSearchCV(mlp, {
            'alpha': [100, 10, 1, .01, .001, .0001],
            'max_iter': [2000],
            'hidden_layer_sizes': [[150, 150, 150, 150, 150], [300, 150], [300, 100, 50], [100, 100, 100, 100]],
            }, cv=10)
grid.fit(X_train, y_train)
best_mlp = grid.best_estimator_
print(grid.best_score_)
print(grid.best_params_)

0.727668845316
{'alpha': 1, 'hidden_layer_sizes': [300, 100, 50], 'max_iter': 2000}


In [30]:
accuracy = []
accuracy_train = []
for i in range (100):

    X_train, X_test, y_train, y_test = train_test_split(data[best_features], data['Pus'], 
                                                        test_size=0.30)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    best_mlp.fit(X_train, y_train)
    predictions = best_mlp.predict(X_test)
    predictions_train = best_mlp.predict(X_train)
    accuracy.append(accuracy_score(y_test, predictions))
    accuracy_train.append(accuracy_score(y_train, predictions_train))

print(np.mean(accuracy), np.mean(accuracy_train))


0.739949238579 0.747189542484


Superior to Random Forest

This is leftover code from other explorations

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

def model_selector(df, features):
    
    models = [{
        "name": 'K Neighbors Classifier', 
        'estimator':KNeighborsClassifier(),
        'hyperparameters': {
            'n_neighbors': range(1,20,2),
            'weights': ['distance', 'uniform'],
            'algorithm': ['ball_tree', 'kd_tree', 'brute'],
            'p': [1,2]
        }
    }, 
    {
        "name": 'Logistic Regression',
        'estimator':LogisticRegression(),
        'hyperparameters': {
            "solver": ["newton-cg", "lbfgs", "liblinear"], 
        }
        
    }, 
    {
        'name': "Random Forest Classifier",
        'estimator':RandomForestClassifier(), 
        'hyperparameters': {
            "n_estimators": [10, 50, 200],
            "criterion": ["entropy", "gini"],
            "max_depth": [2, 5, 10],
            "max_features": ["log2", "sqrt"],
            "min_samples_leaf": [1, 5, 8],
            "min_samples_split": [2, 3, 5]
        }
    }]
    for model in models:
        print(model['name'], "\n_____________\n")
        grid = GridSearchCV(model["estimator"], 
                            param_grid=model["hyperparameters"], 
                           cv = 10)
        grid.fit(X_train, y_train)
        model['best model'] = grid.best_estimator_
        model['best parameters'] = grid.best_params_
        model['best score'] = grid.best_score_
        print('best score:',grid.best_score_)
        print('best parameters:', grid.best_params_)
        print("\n\n")
    return models
best_models = model_selector(data, best_features)
        
    

K Neighbors Classifier 
_____________

best score: 0.758169934641
best parameters: {'algorithm': 'ball_tree', 'n_neighbors': 11, 'p': 1, 'weights': 'distance'}



Logistic Regression 
_____________

best score: 0.751633986928
best parameters: {'solver': 'newton-cg'}



Random Forest Classifier 
_____________

best score: 0.762527233115
best parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 5, 'min_samples_split': 3, 'n_estimators': 10}





In [25]:
from sklearn.metrics import classification_report
rf = RandomForestClassifier(max_depth=2, max_features='log2', min_samples_leaf=5, 
                            min_samples_split=2, n_estimators=200)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
predictions_train = rf.predict(X_train)
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions), accuracy_score(y_train, predictions_train))

             precision    recall  f1-score   support

        0.0       0.82      0.36      0.50        74
        1.0       0.71      0.95      0.82       123

avg / total       0.75      0.73      0.70       197

0.730964467005 0.738562091503


## Predicting Tonsillectomy

In [26]:
corr_tonsillectomy = data.corr()
(corr_tonsillectomy['Tonsillectomy']).sort_values(ascending=False)

Tonsillectomy            1.000000
Age_cat_Teenager         0.201513
Worsening of Symptoms    0.092711
Dysphagia                0.059006
Pus                      0.049337
Duration Sxs (days)      0.041409
Anorexia                 0.023301
WBC                      0.016059
Trismus                  0.013276
Fever                    0.008162
Otalgia                 -0.004432
Cough                   -0.036175
Gender                  -0.048185
Age_cat_Young Adult     -0.070460
Age_cat_Adult           -0.072168
Age_cat_50+             -0.097153
Name: Tonsillectomy, dtype: float64

In [27]:
best_tonsil_model = model_selector(data, best_features, 'Pus')


TypeError: model_selector() takes 2 positional arguments but 3 were given

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[best_features], data['Tonsillectomy'], test_size=0.33, random_state=42)
X_train = normalize(X_train)
X_test = normalize(X_test)

rf = RandomForestClassifier(criterion='gini', max_depth=10, max_features='log2', min_samples_leaf=20, 
                            min_samples_split=2, n_estimators=4, class_weight='balanced')
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
predictions_train = rf.predict(X_train)
accuracy = accuracy_score(y_test, predictions)
accuracy_train = accuracy_score(y_train, predictions_train)
print(accuracy_train, accuracy)
predictions

In [ ]:
print(classification_report(y_test, predictions))


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPClassifier(max_iter = 2000)
mlp.fit(X_train, y_train)
predictions = mlp.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy

In [ ]:
accuracy = []
accuracy_train = []
for i in range (150):

    X_train, X_test, y_train, y_test = train_test_split(data[best_features], data['Pus'], 
                                                        test_size=0.20, random_state=i)
    X_train = normalize(X_train)
    X_test = normalize(X_test)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    mlp = MLPClassifier(max_iter = 2000, alpha=3)
    mlp.fit(X_train, y_train)
    predictions = mlp.predict(X_test)
    predictions_train = mlp.predict(X_train)
    accuracy.append(accuracy_score(y_test, predictions))
    accuracy_train.append(accuracy_score(y_train, predictions_train))
    #print(classification_report(y_test, predictions))

print(np.mean(accuracy), np.mean(accuracy_train))
